In [ ]:
# Goal 
    # - [ ]  Store the text in a vector store
    # - [ ]  Embed it
    # - [ ]  Create a chatbot over that content
    # - [ ]  fine-tune a smaller open-source model on the extracted data

In [ ]:
import os
import faiss
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
from sentence_transformers import SentenceTransformer
RESPONSE_PATH = '/home/linux-pc/gh/projects/NeuralNexus/New-Features/CustomLLM/custom-voice-bot/data/prompt_response/response1.json'
import json

In [6]:
with open(RESPONSE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [7]:
texts = data["text"]

In [8]:
# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all texts
embeddings = model.encode(data["text"], convert_to_tensor=True)

/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Store embeddings in a vector store

# Convert embeddings to numpy array (float32)
embeddings_np = embeddings.cpu().numpy().astype('float32')

# Dimension of embeddings
d = embeddings_np.shape[1]

# Create FAISS index
index = faiss.IndexFlatL2(d)  # L2 distance metric

# Add embeddings to index
index.add(embeddings_np)

# Save index if needed
faiss.write_index(index, "texts.index")


In [10]:
def search_similar(query, index, model, texts, k=3):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy().astype('float32')
    distances, indices = index.search(query_embedding, k)
    results = [texts[i] for i in indices[0]]
    return results

In [ ]:
query = "Hey I miss you a lot. I love you, mom."

Top relevant texts: ['I love YOU!!!', 'I love you too sweetie.... Thanks for your help!', 'Love you both ..... I am fine']


In [17]:
 
results = search_similar(query, index, model, texts)
print("Top relevant texts:", results)

Top relevant texts: ['I love YOU!!!', 'I love you too sweetie.... Thanks for your help!', 'Love you both ..... I am fine']


In [13]:
# Wrap your texts
dataset = Dataset.from_dict({"text": texts})

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model_ft = GPT2LMHeadModel.from_pretrained("gpt2")
model_ft.resize_token_embeddings(len(tokenizer))

/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding(50257, 768)

In [15]:
# Tokenize
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)


In [11]:
tokenized_dataset = dataset.map(tokenize)

Map: 100%|██████████| 45/45 [00:00<00:00, 3241.07 examples/s]


In [12]:
# Fine-tune

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=10,
    prediction_loss_only=True,
    fp16=True,  # Set to True if using GPU
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=tokenized_dataset,
)


/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/accelerate/accelerator.py:477: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [13]:

trainer.train()


  0%|          | 0/69 [00:00<?, ?it/s]

AttributeError: 'AdamW' object has no attribute 'train'

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_ft = GPT2LMHeadModel.from_pretrained("gpt2")

# Load and tokenize dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")

def tokenize_function(examples):
    return tokenizer(examples["text"], return_tensors="pt", truncation=True, padding="max_length", max_length=128)

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=10,
    prediction_loss_only=True,
    fp16=True,  # Set to True if using GPU
    remove_unused_columns=False,
)

# Create trainer
trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train
trainer.train()



/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 367/367 [00:00<00:00, 3959.23 examples/s]
/home/linux-pc/anaconda3/envs/custom-voice-bot/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.